In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['EAG']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('electroantennogram', 59),
 ('ether a go go', 10),
 ('experimental autoimmune glomerulonephritis', 7),
 ('extract of angelica gigas', 5),
 ('ether linked acylglycerol', 5)]

In [6]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

INFO: [2020-09-25 14:47:31] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.2/bio_ontology.pkl


In [6]:
result = [grounding_map, names, pos_labels]

In [7]:
result

[{'experimental autoimmune glomerulonephritis': 'DOID:DOID:0040094',
  'ether linked acylglycerol': 'CHEBI:CHEBI:17408'},
 {'DOID:DOID:0040094': 'autoimmune glomerulonephritis',
  'CHEBI:CHEBI:17408': 'monoacylglycerol'},
 []]

In [8]:
d = load_disambiguator('EAG')
result = [d.grounding_dict['EAG'], d.names, d.pos_labels]
result

[{'1 o hexadecyl 2 acetylglycerol': 'CHEBI:CHEBI:75882',
  'electroantennogram': 'electroantennogram',
  'electroantennographic': 'electroantennogram',
  'electroantennography': 'electroantennogram',
  'ethanol extract of angelica gigas': 'ungrounded',
  'ether a go go': 'FPLX:KCNH',
  'ether à go go': 'FPLX:KCNH',
  'experimental autoimmune gastritis': 'ungrounded',
  'experimental autoimmune glomerulonephritis': 'ungrounded',
  'octadecenyl 2 acetylglycerol': 'CHEBI:CHEBI:75882'},
 {'CHEBI:CHEBI:75882': '1-alkyl-2-acetylglycerol',
  'electroantennogram': 'electroantennogram',
  'FPLX:KCNH': 'KCNH'},
 ['CHEBI:CHEBI:75882', 'FPLX:KCNH']]

In [8]:
grounding_map, names, pos_labels = [{'1 o hexadecyl 2 acetylglycerol': 'CHEBI:CHEBI:75882',
  'electroantennogram': 'electroantennogram',
  'electroantennographic': 'electroantennogram',
  'electroantennography': 'electroantennogram',
  'ethanol extract of angelica gigas': 'ungrounded',
  'ether a go go': 'FPLX:KCNH',
  'ether à go go': 'FPLX:KCNH',
  'experimental autoimmune gastritis': 'ungrounded',
  'experimental autoimmune glomerulonephritis': 'ungrounded',
  'octadecenyl 2 acetylglycerol': 'CHEBI:CHEBI:75882'},
 {'CHEBI:CHEBI:75882': '1-alkyl-2-acetylglycerol',
  'electroantennogram': 'electroantennogram',
  'FPLX:KCNH': 'KCNH'},
 ['CHEBI:CHEBI:75882', 'FPLX:KCNH']]

In [9]:
excluded_longforms = []

In [10]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [11]:
additional_entities = {}

In [12]:
unambiguous_agent_texts = {'FPLX:KCNH': ['KCNH', ['KCNH']]}

In [13]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [14]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [15]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [16]:
intersection1

[]

In [17]:
intersection2

[]

In [20]:
len(corpus)

57

In [19]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [21]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [22]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-12-01 01:39:10] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-12-01 01:39:12] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9333333333333332 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [23]:
classifier.stats

{'label_distribution': {'electroantennogram': 41,
  'ungrounded': 7,
  'FPLX:KCNH': 9},
 'f1': {'mean': 0.933333, 'std': 0.133333},
 'precision': {'mean': 1.0, 'std': 0.0},
 'recall': {'mean': 0.9, 'std': 0.2},
 'electroantennogram': {'f1': {'mean': 0.988235, 'std': 0.023529},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 0.977778, 'std': 0.044444}},
 'ungrounded': {'f1': {'mean': 1.0, 'std': 0.0},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'FPLX:KCNH': {'f1': {'mean': 0.933333, 'std': 0.133333},
  'pr': {'mean': 0.9, 'std': 0.2},
  'rc': {'mean': 1.0, 'std': 0.0}}}

In [95]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [96]:
disamb.dump(model_name, results_path)

In [97]:
print(disamb.info())

Disambiguation model for EAG

Produces the disambiguations:
	1-alkyl-2-acetylglycerol*	CHEBI:CHEBI:75882
	KCNH*	FPLX:KCNH
	electroantennogram	electroantennogram

Class level metrics:
--------------------
Grounding               	Count	F1     
      electroantennogram	41	0.98824
                    KCNH*	 9	0.93333
              Ungrounded	 7	    1.0

Weighted Metrics:
-----------------
	F1 score:	0.93333
	Precision:	1.0
	Recall:		0.9

* Positive labels
See Docstring for explanation



In [98]:
model_to_s3(disamb)